# Simon algorithm（Overview）

We explain Simon algorithm algorithm.

In this algorithm, for a function $f_s(x)$ with $n$-bit input and output ($s$ is any $n$-bit sequence), one of the following is assumed to be true.

Case1: Always return different outputs for different inputs (one-to-one correspondence)

Case2: For input $x, x'$, if $x' = x\oplus s$, then $f_s(x) = f_s(x')$. That is, it returns the same output for two inputs.

This algorithm determines whether Oracle is case 1 or case 2 above.


The concrete quantum circuit is as follows. The contents of $U_f$ are shown for the case of case 2 above and $s=1001$.   
The number of qubits is $2 n$.

<img src="../img/103_img.png" width="50%">

Check the state.

$$
\begin{align}
\lvert \psi_1\rangle &= \frac{1}{\sqrt{2^n}} \biggl(\otimes^n H\lvert 0\rangle \biggr) \lvert 0\rangle^{\otimes n} \\
&= \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} \lvert x\rangle \lvert 0\rangle^{\otimes n}
\end{align}
$$

Next, consider $\lvert \psi_2 \rangle$.

Here, for $f_s(x)$, we have the following oracle gate $U_f$.

$$
U_f \lvert x \rangle \lvert 0 \rangle = \lvert x \rangle \lvert f_s(x) \rangle
$$

Using this $U_f$, we get

$$
\lvert \psi_2 \rangle = \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} \lvert x\rangle \lvert f_s(x)\rangle
$$

Therefore, $\lvert \psi_3 \rangle$ is as follows

$$
\lvert \psi_3 \rangle = \frac{1}{2^n} \sum_{x=0}^{2^n-1}\sum_{y=0}^{2^n-1} (-1)^{x\cdot y} \lvert y\rangle \lvert f_s(x)\rangle
$$

Now, consider what the measurement result of $\lvert y \rangle$ would be if $f_s(x)$ were as follows.

Case 1: Always return different outputs for different inputs (one-to-one correspondence)

All measurement results are obtained with equal probability.

Case 2: For input $x, x'$, if $x' = x\oplus s$, then $f_s(x) = f_s(x')$. That is, it returns the same output for two inputs.

Notice the amplitude $A(y, x)$ of the state $\lvert y \rangle \lvert f_s(x) \rangle = \lvert y \rangle \lvert f_s(x\oplus s) \rangle$.

$$
A(y, x) = \frac{1}{2^n} \{(-1)^{x\cdot y} + (-1)^{(x\oplus s) \cdot y}\}
$$

As you can see from the equation, the amplitude of $y$ such that $y\cdot s \equiv 1 \bmod2$  is $0$ due to cancellation.  
Therefore, only $y$ is measured such that $y\cdot s \equiv 0 \bmod2$.

In both case 1 and case 2, if such $n$ different $y$ are obtained by measurement (except for $00. .0$), we can determine $s'$ such that $y\cdot s' \equiv 0 \bmod2$ for all those $y$.
 
In the case 1, $s'$ is completely random.  
However, in the case 2, $f_s(s') = f_s(0)$ is always true because $s' = 0\oplus s'$.

Thus, except for the case where $s'$ such that $f_s(s') = f_s(0)$ is obtained from case 1 with probability $1 / 2^n$, we can check whether $s'$ is obtained from case 1 or case 2 using the oracle gate.  
The oracle can be determined from the above.

Finally, we consider the implementation of the oracle gate $U_f$.

In the case 1, it is enough that the output has a one-to-one correspondence with the input $x$.  
For simplicity, let's consider a circuit that randomly inserts an $X$ gate.

The case 2 is a bit more complicated.  
First, the $CX$ gate creates the following state.

$$
\lvert \psi_{1a} \rangle = \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} \lvert x\rangle \lvert x\rangle
$$

Next, for the lowest index $i'$ where $s_i=1$, take XOR of the auxiliary register and $s$ only if $x_{i'} = 0$.  
As a result, we get the following $\lvert\psi_2\rangle$.

$$
\begin{align}
\lvert \psi_{2} \rangle &= \frac{1}{\sqrt{2^n}} \biggl(\sum_{\{x_{i'}=0\}} \lvert x\rangle \lvert x \oplus s\rangle + \sum_{\{x_{i'}=1\}} \lvert x\rangle \lvert x\rangle \biggr) \\
&= \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} \lvert x\rangle \lvert f_s(x)\rangle
\end{align}
$$

We can confirm that $f_s(x)$ satisfies case 2 by calculation.

Let's implement this with blueqat.

In [178]:
from blueqat import Circuit
import numpy as np

Prepare a function for two types of oracle gates $U_f$.

In [179]:
def oracle_1(c, s):
    _n = len(s)
    for i in range(_n):
        if np.random.rand() > 0.5:
            c.x[i]
    for i in range(_n):
        c.cx[i, i + _n]
        
def oracle_2(c, s):
    _n = len(s)
    flag = 0
    for i, si in enumerate(reversed(s)):
        c.cx[i, i + _n]
        if si == '1' and flag == 0:
            c.x[i]
            for j, sj in enumerate(s):
                if sj == '1':
                    c.cx[i, j + _n]
            c.x[i]
            flag = 1

The following is the main body of the algorithm.  
First, use a random number to determine the oracle (one of the two types) and the $s$ you want to find.  
(In the following, the values are fixed to reproduce the quantum circuit shown in the figure above.)

In [180]:
n = 4
N = np.random.randint(1, 2**n-1)
s = bin(N)[2:].zfill(n)
which_oracle = np.random.rand()

### to reproduce the quantum circuit shown in the figure above ###
### Erasing these two lines will randomly determine s and oracle###
s = "1001" 
which_oracle = 0
######

c = Circuit(n * 2)
c.h[:n]

if which_oracle > 0.5:
    oracle_1(c, s)
    oracle = "oracle 1"
else:
    oracle_2(c, s)
    oracle = "oracle 2"
    
c.h[:n].m[:n]
res = c.run(shots = 1000)

In [181]:
res

Counter({'10010000': 111,
         '00100000': 120,
         '10110000': 128,
         '11110000': 123,
         '01100000': 137,
         '01000000': 133,
         '00000000': 115,
         '11010000': 133})

Extract $n$ results other than '00...0' from the sampling result.  

In [182]:
res_list = list(res.keys())
_res_list = []
for i in res_list:
    if i[:n] != '0'*4:
        _res_list.append(i[:n])
    if len(_res_list) == 4:
        break
            
print(_res_list)

['1001', '0010', '1011', '1111']


Find $s'$ from the extracted result.  
(Here, we are simply looking for $s'$ that matches the condition by brute force, but it is possible to find it efficiently in linear algebra.)

If the oracle in case 2 is selected, the resulting $s'$ should be equal to $s$.

In [185]:
for i in range(2**n):
    l = bin(i)[2:].zfill(n)
    flag = 1
    for sampled in _res_list: 
        mod = np.sum(np.array(list(l), dtype = np.int64) * np.array(list(sampled), dtype = np.int64)) % 2
        if mod:
            flag = 0
            break
    if flag:
        output_s = l

In [186]:
print("s' =", output_s)
print("s =", s)

s' = 1001
s = 1001
